# Recommender system for books: Modelisation

In [1]:
# Data manipulation
import pandas as pd 
import numpy as np 
import gc

# Graphical libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Modelisation libraries
from surprise import BaselineOnly, SVD, NMF, KNNBasic, KNNWithMeans, KNNWithZScore
from surprise import accuracy
from surprise.model_selection import train_test_split, GridSearchCV
from surprise import Dataset
from surprise import Reader

from sklearn.metrics.pairwise import cosine_similarity, linear_kernel
from sklearn.metrics import mean_squared_error
from sklearn.feature_extraction.text import TfidfVectorizer

# GHG Emissions
from codecarbon import EmissionsTracker

In [2]:
path = 'Datasets/'

In [3]:
# Importing data about users and books 
users = pd.read_parquet(path+"users_data_cleaning.parquet.gzip")
books = pd.read_parquet(path+"books_data_cleaning.parquet.gzip")

In [4]:
books.rename(columns={"Rating" : "Average_Rating"},
             inplace=True)

It exists different kinds of recommender engines. In this proof of concept, simple recommender, user based collaborative filtering, content-based recommender and hybrid methods will be testing to find the best recommender engines. 

## 1) Popularity based

Simple recommender is the most basic recommender engines. It based on popularity. To calcule it, rating and number of reviews are used. The formula used is detailled in Bhowmick et al. (2021).

In [5]:
# creating a new dataset for the calculus
simple_reco = books.copy()
simple_reco = simple_reco[["Id", "Name", "Authors",
                           "Average_Rating", "CountsOfReview"]]
simple_reco.head()

,Id,Name,Authors,Average_Rating,CountsOfReview
0,4000063,The Flintstones in Viva Rock Vegas,Ellen Miles,3.82,1
1,4000100,Little Rhody,Neta Lohnes Frazier,4.33,1
2,4000228,Finance And Investments Using The Wall Street ...,Peter R. Crabb,0.00,0
3,4000366,Shorty McCabe Looks 'Em Over,Sewell Ford,0.00,0
4,4000441,Plant Pathology,George N. Agrios,4.52,0


In [6]:
# calculating the average rating
C = simple_reco['Average_Rating'].mean()
print(C)

3.7320950652695215


In [7]:
# calculating and keeping
# books with the 90th percentile
m = simple_reco['CountsOfReview'].quantile(0.90)
print(m)

41.0


In [8]:
# selecting only books with at least 41 reviews
q_books = simple_reco.copy().loc[simple_reco['CountsOfReview'] >= m]
q_books.shape

(12116, 5)

In [9]:
def weighted_rating(x, m=m, C=C):
    v = x['CountsOfReview']
    R = x['Average_Rating']
    return (v/(v+m) * R) + (m/(m+v) * C)

In [10]:
q_books['score'] = q_books.apply(weighted_rating, axis=1)

In [11]:
#Sort movies based on score calculated above
q_books = q_books.sort_values('score',
                              ascending=False)

#Print the top 15 books
q_books[["Id",
         "Name",
         "Authors",
         "CountsOfReview", 
         "Average_Rating", 
         "score"]].head(20)

,Id,Name,Authors,CountsOfReview,Average_Rating,score
44289,862041,"Harry Potter Series Box Set (Harry Potter, #1-7)",J.K. Rowling,6522,4.74,4.733703
40413,818056,Harry Potter and the Deathly Hallows (Harry Po...,J.K. Rowling,952,4.62,4.583339
115327,1215032,"The Wise Man's Fear (The Kingkiller Chronicle,...",Patrick Rothfuss,16523,4.57,4.567926
61446,3165162,Percy Jackson and the Olympians (Percy Jackson...,Rick Riordan,546,4.59,4.530078
72565,1025685,"The Absolute Sandman, Volume Two",Neil Gaiman,198,4.69,4.525673
91715,2495562,The Wise Man's Fear (The Kingkiller Chronicle...,Patrick Rothfuss,488,4.56,4.495833
82953,2186848,"The Absolute Sandman, Volume Three",Neil Gaiman,140,4.71,4.488486
98149,2767793,"The Hero of Ages (Mistborn, #3)",Brandon Sanderson,10101,4.49,4.486936
30441,1179967,"A Storm of Swords (A Song of Ice and Fire, #3)",George R.R. Martin,583,4.54,4.486917
66035,3362870,"The Hero of Ages (Mistborn, #3)",Brandon Sanderson,1289,4.49,4.466636


In [12]:
print(round(np.sqrt(mean_squared_error(q_books['Average_Rating'],
                                 q_books["score"])), 3))

0.108


In [13]:
del q_books, simple_reco
gc.collect()

0

This method had disadvantage to make recommendation based on the most popular books. Books with few ratings have less change to be recommend. 

## 2) User-based collaborative filtering

### a) Preparing the data for Surprise library

In [14]:
data = pd.merge(users, books, on = "Id")
data.rename(columns={"Name" : "Title",
                     "Id" : "Book_Id"}, 
                     inplace=True)

In [15]:
reader = Reader(rating_scale=(1, 5))
data_surprise = Dataset.load_from_df(data[['User_Id', 'Book_Id', 'Rating']], reader)

In [16]:
trainset, testset = train_test_split(data_surprise, test_size=.2)

In [17]:
bsl_options = {'method': 'als',
               'n_epochs': 5,
               'reg_u': 12,
               'reg_i': 5
               }
base_als = BaselineOnly(bsl_options=bsl_options)
predictions = base_als.fit(trainset).test(testset)

Estimating biases using als...


In [18]:
del predictions
gc.collect()

0

In [19]:
bsl_options = {'method': 'sgd',
               'learning_rate': .00005,
               }
base_sgd = BaselineOnly(bsl_options=bsl_options)
predictions = base_sgd.fit(trainset).test(testset)

Estimating biases using sgd...


In [20]:
del predictions
gc.collect()

0

### b) Matrix Factorization based algorithms

In [21]:
svd = SVD()
nmf = NMF()

In [22]:
model = [svd, nmf]
dict_model = {}

for model in model:
  y_pred = model.fit(trainset).test(testset)
  rmse = accuracy.rmse(y_pred)
  dict_model[model] = (rmse.round(3))

eval_model = pd.DataFrame(dict_model, index=["RMSE"])
eval_model.columns = ["SVD", "NMF"]

eval_model

RMSE: 0.7622
RMSE: 0.8725


,SVD,NMF
RMSE,0.762,0.872


In [ ]:
param_grid = {'n_factors': [50, 75, 100, 125],
              'n_epochs': [5, 10, 15, 20, 25], 
              'lr_all': [0.001, 0.002, 0.005, 0.1],
              'reg_all': [0.1, 0.2, 0.4, 0.6],
              'random_state' : [42]
}

gs = GridSearchCV(SVD,
                  param_grid,
                  measures=['rmse', 'mae'],
                  cv=5)

gs.fit(data_surprise)

# best RMSE score
print(gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

In [ ]:
tracker = EmissionsTracker()
svd_gs = gs.best_estimator['rmse']

tracker.start()
predictions = svd_gs.fit(trainset).test(testset)
emissions = tracker.stop()

accuracy.rmse(predictions)
print("GHG emission:", emissions)

In [ ]:
del predictions,\
    gs,\
    dict_model,\
    emissions,\
    tracker
gc.collect()

### c) K-NN based models

In [23]:
# Initializing similarities options
sim_options = {'name': 'cosine',
               'user_based': True
               }

In [24]:
knnbasic = KNNBasic()
knnmeans = KNNWithMeans()
knnzscore = KNNWithZScore()

In [25]:
model = [knnbasic, knnmeans, knnzscore]
dict_model = {}
tracker = EmissionsTracker()

for model in model:
    y_pred = model.fit(trainset).test(testset)
    rmse = accuracy.rmse(y_pred)
    dict_model[model] = (rmse.round(3))

eval_model = pd.DataFrame(dict_model, index=["RMSE"])
eval_model.columns = ["KNN Basic", "KNN Means", "KNN Z-Score"]

eval_model

Computing the msd similarity matrix...


/home/cecile/anaconda3/envs/nlpenv/lib/python3.9/site-packages/apscheduler/util.py:95: PytzUsageWarning: The zone attribute is specific to pytz's interface; please migrate to a new time zone provider. For more details on how to do so, see https://pytz-deprecation-shim.readthedocs.io/en/latest/migration.html
  if obj.zone == 'local':


Done computing similarity matrix.
RMSE: 0.7676
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.7679
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.7611


,KNN Basic,KNN Means,KNN Z-Score
RMSE,0.768,0.768,0.761


In [26]:
param_grid = {'k': [20, 30, 40, 50, 60, 70, 80]}

gs = GridSearchCV(KNNWithZScore,
                  param_grid,
                  measures=['rmse', 'mae'],
                  cv=5)

gs.fit(data_surprise)

# best RMSE score
print(gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computi

In [27]:
knn_gs = gs.best_estimator['rmse']

tracker = EmissionsTracker()

tracker.start()
predictions = knn_gs.fit(trainset).test(testset)
emissions = tracker.stop()


accuracy.rmse(predictions)
print("GHG emission:", emissions)

/home/cecile/anaconda3/envs/nlpenv/lib/python3.9/site-packages/apscheduler/util.py:95: PytzUsageWarning: The zone attribute is specific to pytz's interface; please migrate to a new time zone provider. For more details on how to do so, see https://pytz-deprecation-shim.readthedocs.io/en/latest/migration.html
  if obj.zone == 'local':
/home/cecile/anaconda3/envs/nlpenv/lib/python3.9/site-packages/apscheduler/triggers/interval.py:66: PytzUsageWarning: The normalize method is no longer necessary, as this time zone supports the fold attribute (PEP 495). For more details on migrating to a PEP 495-compliant implementation, see https://pytz-deprecation-shim.readthedocs.io/en/latest/migration.html
  return self.timezone.normalize(next_fire_time)


Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.7426
GHG emission: 2.0916046788691988e-05


In [28]:
del gs,\
    dict_model,\
    predictions,\
    tracker,\
    emissions
gc.collect()

82

## 3) Content-based algorithms

### a) With Tfidf

https://medium.com/analytics-vidhya/content-based-recommender-systems-in-python-2b330e01eb80

In [29]:
desc = books.copy()
desc = desc[["Id", "Name", "Authors", "Description"]]

In [30]:
desc = desc.sample(frac=.2,
                   random_state=42)

In [31]:
desc.reset_index(inplace=True)
desc.drop("index", axis=1, inplace=True)

In [32]:
tracker = EmissionsTracker()

tracker.start()
tfidf = TfidfVectorizer(stop_words='english')
desc['Description'] = desc['Description'].fillna("")
descr_matrix = tfidf.fit_transform(desc['Description'])
similarity_matrix = linear_kernel(descr_matrix, descr_matrix)
emissions = tracker.stop()

print("GHG Emissions:", emissions)

GHG Emissions: 2.3392958143540964e-05


In [33]:
mapping = pd.Series(desc.index,
          index = desc['Name'])
print(mapping)

Name
On the Other Side of Mount Ararat: A Story of a Vanished City        0
The Tall Uncut: Stories                                              1
Heart Essence of the Vast Expanse: A Story of Transmission           2
Don't Look a Ghost Horse in the Mouth                                3
Turpentine                                                           4
                                                                 ...  
Infernal Revenue (The Destroyer, #96)                            23911
Die with Me                                                      23912
Kingyo: The Artistry of the Japanese Goldfish                    23913
The Shadow Roads (The Swans' War, book 3)                        23914
Roommates                                                        23915
Length: 23916, dtype: int64


In [34]:
def recommend_books(book_input):
    print("Book Read: ", book_input)
    book_index = mapping[book_input]
    #get similarity values with other books
    #similarity_score is the list of index and similarity matrix
    similarity_score = list(enumerate(similarity_matrix[book_index]))
    #sort in descending order the similarity score of movie inputted with all the other books
    similarity_score = sorted(similarity_score, key=lambda x: x[1], reverse=True)
    
    # Get the scores of the 20 most similar books. Ignore the first book.
    similarity_score = similarity_score[1:20]
    
    #return book names using the mapping series
    book_indices = [i[0] for i in similarity_score]
    return (desc['Name'].iloc[book_indices])

In [35]:
recommend_books('Pet Sematary')

Book Read:  Pet Sematary


3230                                  Black Cat, Volume 11
15651    Glimpses of Maine's Angling Past (Images of Am...
18878                        Oh, the Thinks You Can Think!
14474                           Go to the Room of the Eyes
663      Boston and the American Revolution: Boston Nat...
9972                                                   Ash
10932               True Blue (Sweet Valley Jr. High, #18)
18362                                            Bold Wolf
8047     African American Life in the Rural South, 1900...
7174                                       White House Q&A
17721                                        The Long Life
8842     The Horizontal World: Growing Up Wild in the M...
15942                      That Yankee Cat: The Maine Coon
6775                                        The Best Story
22298                                       Mansfield Park
9071                                    The Good Neighbour
21028                                             The We

In [36]:
del tracker, emissions
gc.collect()

32

### b) Using Transformers

https://towardsdatascience.com/hands-on-content-based-recommender-system-using-python-1d643bf314e4

In [ ]:
from sentence_transformers import SentenceTransformer
import pandas as pd
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA

In [ ]:
data = pd.read_parquet(path+"books_with_complete_description.parquet.gzip")
data = data[["Name", "Authors", "Description"]].sample(frac=.3, random_state=42)
data.reset_index(inplace=True)
data.drop("index", axis=1, inplace=True)
data.head()

In [ ]:
X = np.array(data["Description"])

In [ ]:
text_data = X

tracker = EmissionsTracker()
tracker.start()

model = SentenceTransformer('distilbert-base-nli-mean-tokens')
embeddings = model.encode(text_data, show_progress_bar=True)
X = np.array(embeddings)
cos_sim_data = pd.DataFrame(cosine_similarity(X))

emissions = tracker.stop()
print("GHG Emissions:", emissions)

In [ ]:
n_comp = 5
pca = PCA(n_components=n_comp)
pca.fit(X)
pca_data = pd.DataFrame(pca.transform(X))
pca_data.head()

In [ ]:
sns.pairplot(pca_data)

In [ ]:
data.loc[data["Authors"] == "Stephen King"]

In [ ]:
def give_recommendations(index, print_recommendation = False,print_recommendation_plots= False):
  index_recomm =cos_sim_data.loc[index].sort_values(ascending=False).index.tolist()[1:6]
  books_recomm =  data['Name'].loc[index_recomm].values
  result = {'Books':books_recomm,'Index':index_recomm}
  if print_recommendation==True:
    print('Book Read: %s \n'%(data['Name'].loc[index]))
    k=1
    for book in books_recomm:
      print('Recommended book #%i : %s \n'%(k, book))
      k = k+1
  if print_recommendation_plots==True:
    print('Book Read - Description:\n %s \n'%(data['Description'].loc[index]))
    k=1
    for q in range(len(books_recomm)):
      plot_q = data['Description'].loc[index_recomm[q]]
      print('Recommended Book #%i - Description:\n %s \n'%(k,plot_q))
      k=k+1
  return result

In [ ]:
give_recommendations(19590, True, True)

In [ ]:
recomm_list = []
for i in range(len(X)):
  recomm_i = give_recommendations(i)
  recomm_list.append(recomm_i['Books'])
recomm_data = pd.DataFrame(recomm_list,columns=['First Recommendation','Second Recommendation','Third Recommendation','Fourth Recommendation','Fifth Recommendation'])
recomm_data['Books Read'] = data['Name']
recomm_data = recomm_data[['Books Read','First Recommendation','Second Recommendation','Third Recommendation','Fourth Recommendation','Fifth Recommendation']]

In [ ]:
recomm_data.sample(frac=1).head()

In [ ]:
del users,\
    books,\
    data
gc.collect()